In [1]:
import dask.dataframe as dask_pd
import dask_geopandas as dask_gpd
import geopandas as gpd
import numpy as np
from shapely.geometry import Polygon
import shapely

/home/nivag/2023 - Linux/.python-3.10-1/lib/python3.10/site-packages/geopandas/_compat.py:124: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
# dask_df_mdt =  dask_pd.read_csv('Compile-MDT/mdt*.csv')
dask_df_mdt =  dask_pd.read_csv('Compile-MDT/mdt*.csv', usecols=[0,1,2,3,4,6,7,8,9], assume_missing=True)

In [3]:
dask_df_mdt

,date,hour,site,enodebid,ci,longitude,latitude,altitude,rsrp_serving
npartitions=909,,,,,,,,,
,float64,float64,object,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...


In [4]:
dask_df_mdt['pointer'] = "POINT (" + dask_df_mdt['longitude'].astype(str) + " " + dask_df_mdt['latitude'].astype(str) + ")"

In [5]:
dask_df_mdt.head(5)

,date,hour,site,enodebid,ci,longitude,latitude,altitude,rsrp_serving,pointer
0,20230502.0,0.0,BH4431339E,430343.0,4.0,106.785127,-6.278504,0.0,-99.0,POINT (106.785126541272 -6.278504133224487)
1,20230502.0,0.0,BH4431339E,430343.0,4.0,106.784161,-6.281347,52.0,-76.0,POINT (106.78416094591152 -6.281347274780273)
2,20230502.0,0.0,BH4431339E,430343.0,5.0,106.762253,-6.300466,0.0,-78.0,POINT (106.7622526600662 -6.300466060638428)
3,20230502.0,0.0,BH4431339E,430343.0,4.0,106.786006,-6.279867,67.0,-98.0,POINT (106.78600630593375 -6.279866695404053)
4,20230502.0,0.0,BH4431339E,430343.0,4.0,106.786006,-6.279867,67.0,-95.0,POINT (106.78600630593375 -6.279866695404053)


In [6]:
dask_gdf_mdt = dask_gpd.from_dask_dataframe(dask_df_mdt, geometry=dask_df_mdt["pointer"].map_partitions(
        gpd.GeoSeries.from_wkt, meta=gpd.GeoSeries([])),)

dask_gdf_mdt = dask_gdf_mdt.set_crs(4326)

In [7]:
# Create Rectangle Grid 100x100 metre from mininum/maximum long-lat in CRS:3426
# These projection are imagined earth as flat object. these are not accurate for higher degree of long-lat
# Based on plotting in QGIS, the difference of grid reach 50 meter in latitude and 25 meter in longitude
# Try to convert them to EPSG:3857 

xmin, ymin, xmax, ymax = dask_gdf_mdt.total_bounds

length =  0.0008985 * 2.5 ### (the distance for 100 metres, so if you want to get 250 metres, need to multiply 2.5)
wide =  0.0008985 * 2.5 ### (the distance for 100 metres, so if you want to get 250 metres, need to multiply 2.5)

cols = list(np.arange(xmin, xmax + wide, wide))
rows = list(np.arange(ymin, ymax + length, length))

polygons = []
for x in cols[:-1]: ### [:-1] these are used stop -1 of position from the end of list 
    for y in rows[:-1]:
        polygons.append(Polygon([(x,y), (x+wide, y), (x+wide, y+length), (x, y+length)]))

grid = gpd.GeoDataFrame({'geometry':polygons}, crs=4326)

grid.reset_index(inplace=True)
grid.rename(columns={'index':'id'}, inplace = True)

In [ ]:
dask_gdf_mdt = dask_gdf_mdt.sjoin(grid, how='inner', predicate='within')

In [ ]:
dask_gdf_mdt.head(10)

In [ ]:
# dask_gdf_mdt['ci'] = dask_gdf_mdt['ci'].astype('category')
# dask_gdf_mdt['ci'] = dask_gdf_mdt['ci'].cat.as_known()
# pivot = dask_gdf_mdt.pivot_table(index='poly_name', columns='ci', values='rsrp_serving', aggfunc='mean')

In [ ]:
result = dask_gdf_mdt.compute()

In [ ]:
result.to_parquet('busy_grid.parquet')